# Distance Matrix computation

In order to compute the distance-dependency of connectivity values corrupted by motion we require a distance matrix which describes inter-nodal distances.

We will compute the distance matrix of a given `surf.gii` and `dlabel.nii` file and output ${P\choose2}$ distances

In [210]:
import os

import numpy as np
import nibabel as nib
import nibabel.cifti2 as cifti

import pandas as pd
import scipy.spatial as sspatial

In [44]:
# Constants
S1200_atlas = "../../data/atlases/S1200_GroupAvg_V1/"
schaefer_atlas = "../../data/atlases/schaefer_2018/"

In [45]:
# Load group template surface
l_surf = nib.load(
    f"{S1200_atlas}/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii"
)

l_va = nib.load(
    f"{S1200_atlas}/S1200.L.midthickness_MSMAll_va.32k_fs_LR.shape.gii"
)

r_surf = nib.load(
    f"{S1200_atlas}/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii"
)

r_va =nib.load(
    f"{S1200_atlas}/S1200.R.midthickness_MSMAll_va.32k_fs_LR.shape.gii"
)

# Load parcellation
schaefer = nib.load(
    f"{schaefer_atlas}/Schaefer2018_200Parcels_17Networks_order.dlabel.nii"
)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [176]:
# Pull LabelAxis
schaefer_lbls = schaefer.get_fdata().astype(int)

# Pull coordinate information
l_coords = l_surf.darrays[0].data
r_coords = r_surf.darrays[0].data
coords = np.vstack((l_coords, r_coords))

# Pull vertex areas
l_areas = l_va.darrays[0].data
r_areas = r_va.darrays[0].data
areas = np.concatenate((l_areas, r_areas))

In [199]:
def area_weighted_centroid(coords, labels, areas):
    '''
    Compute Vertex-area weighted surface centroid
    '''
    
    # Parcel values
    lbls = labels.get_fdata().astype(int)
    
    # Get sort indices for labels
    sort_inds = np.argsort(lbls).flatten()
    v_lbl = labels.header.get_axis(1).vertex[sort_inds]
    
    
    # Get unique labels
    u, ind = np.unique(lbls[:, sort_inds], return_index=True)
    
    # Sort coords and areas
    s_coords = coords[sort_inds, :]
    s_areas = areas[sort_inds]
    
    # Split-em up!
    coord_arr = np.split(s_coords, ind[1:])
    area_arr = np.split(s_areas, ind[1:])
    
    centroids = np.empty((u.shape[0], 3), dtype=float)
    for i, arrs in enumerate(zip(coord_arr, area_arr)):
        c, a = arrs
        centroids[i,:] = (a / a.sum()) @ c
       
    return (u, centroids)
    

In [200]:
u, centroids = area_weighted_centroid(coords, schaefer, areas)

In [208]:
# Pairwise distance matrix
D = sspatial.distance.cdist(centroids, centroids)

In [221]:
# Transform into pandas dataframe
col_names = [
    schaefer.header\
        .get_axis(0).label[0][i][0] for i in u
]
df = pd.DataFrame(D, columns=col_names,
                 index=col_names)\
                .drop('???').drop('???',axis=1)


In [224]:
df = df.where(np.triu(np.ones(df.shape)).astype(np.bool))
df = df.stack().reset_index()
df.columns = ['Row','Column','Value']

<ipython-input-224-0bddf0ed8109>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = df.where(np.triu(np.ones(df.shape)).astype(np.bool))


In [226]:
df.to_parquet('../../data/intermediate/schaefer_200_distance.parquet')